<a href="https://colab.research.google.com/github/linyuehzzz/dp_census/blob/main/sdv_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##**SDV**
This code implements the demo code for SDV.  
Yue Lin (lin.3326 at osu.edu)  
Created: 8/18/2021

In [1]:
pip install sdv

     |████████████████████████████████| 85 kB 1.4 MB/s 
     |████████████████████████████████| 50 kB 2.4 MB/s 
     |████████████████████████████████| 17.9 MB 61.2 MB/s 
     |████████████████████████████████| 1.1 MB 34.2 MB/s 
     |████████████████████████████████| 5.7 MB 21.6 MB/s 
     |████████████████████████████████| 95 kB 4.2 MB/s 
     |████████████████████████████████| 9.5 MB 29.1 MB/s 
     |████████████████████████████████| 22.3 MB 1.7 MB/s 
     |████████████████████████████████| 296 kB 48.2 MB/s 
     |████████████████████████████████| 9.5 MB 39.2 MB/s 
  Attempting uninstall: pandas
    Found existing installation: pandas 1.1.5
    Uninstalling pandas-1.1.5:
      Successfully uninstalled pandas-1.1.5
  Attempting uninstall: statsmodels
    Found existing installation: statsmodels 0.10.2
    Uninstalling statsmodels-0.10.2:
      Successfully uninstalled statsmodels-0.10.2
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 0.22.2.post1
  

####**GaussianCopula Model**

Load original data

In [29]:
from sdv.demo import load_tabular_demo

data = load_tabular_demo('student_placements')
data.head()

,student_id,gender,second_perc,high_perc,high_spec,degree_perc,degree_type,work_experience,experience_years,employability_perc,mba_spec,mba_perc,salary,placed,start_date,end_date,duration
0,17264,M,67.00,91.00,Commerce,58.00,Sci&Tech,False,0,55.0,Mkt&HR,58.80,27000.0,True,2020-07-23,2020-10-12,3.0
1,17265,M,79.33,78.33,Science,77.48,Sci&Tech,True,1,86.5,Mkt&Fin,66.28,20000.0,True,2020-01-11,2020-04-09,3.0
2,17266,M,65.00,68.00,Arts,64.00,Comm&Mgmt,False,0,75.0,Mkt&Fin,57.80,25000.0,True,2020-01-26,2020-07-13,6.0
3,17267,M,56.00,52.00,Science,52.00,Sci&Tech,False,0,66.0,Mkt&HR,59.43,NaN,False,NaT,NaT,NaN
4,17268,M,85.80,73.60,Commerce,73.30,Comm&Mgmt,False,0,96.8,Mkt&Fin,55.50,42500.0,True,2020-07-04,2020-09-27,3.0


Constraints (reject sampling)

In [30]:
from sdv.constraints import GreaterThan

end_date_gt_start_date_constraint = GreaterThan(
    low='start_date',
    high='end_date',
    handling_strategy='reject_sampling'
)

Constraints (transform)

In [43]:
# from sdv.constraints import ColumnFormula
# import numpy as np

# def duration(data):
#     diff = ((data.end_date - data.start_date)/np.timedelta64(1, 'M'))
#     return diff.astype('Int64', errors='ignore')

# duration_constraint = ColumnFormula(
#     column='duration',
#     formula=duration,
#     handling_strategy='transform'
# )

0      2.661246
1      2.924085
2      5.552475
3           NaN
4      2.792665
         ...   
210    2.792665
211    6.373848
212    6.340993
213    3.022649
214         NaN
Length: 215, dtype: float64

Fit original data

In [44]:
from sdv.tabular import GaussianCopula

constraints = [
    end_date_gt_start_date_constraint
]

model = GaussianCopula(constraints=constraints)
model.fit(data)

/usr/local/lib/python3.7/dist-packages/scipy/stats/_continuous_distns.py:547: RuntimeWarning: invalid value encountered in sqrt
  sk = 2*(b-a)*np.sqrt(a + b + 1) / (a + b + 2) / np.sqrt(a*b)
/usr/local/lib/python3.7/dist-packages/scipy/stats/_continuous_distns.py:4798: RuntimeWarning: divide by zero encountered in true_divide
  return c**2 / (c**2 - n**2)
/usr/local/lib/python3.7/dist-packages/scipy/stats/_distn_infrastructure.py:2407: RuntimeWarning: invalid value encountered in double_scalars
  Lhat = muhat - Shat*mu
/usr/local/lib/python3.7/dist-packages/scipy/stats/_continuous_distns.py:4798: RuntimeWarning: divide by zero encountered in true_divide
  return c**2 / (c**2 - n**2)
/usr/local/lib/python3.7/dist-packages/scipy/stats/_distn_infrastructure.py:2407: RuntimeWarning: invalid value encountered in double_scalars
  Lhat = muhat - Shat*mu
/usr/local/lib/python3.7/dist-packages/scipy/stats/_continuous_distns.py:547: RuntimeWarning: invalid value encountered in sqrt
  sk = 2*(b-a

Get distributions

In [33]:
distributions = model.get_distributions()
distributions

{'degree_perc': 'copulas.univariate.student_t.StudentTUnivariate',
 'degree_type#0': 'copulas.univariate.gaussian.GaussianUnivariate',
 'degree_type#1': 'copulas.univariate.student_t.StudentTUnivariate',
 'degree_type#2': 'copulas.univariate.gaussian.GaussianUnivariate',
 'employability_perc': 'copulas.univariate.truncated_gaussian.TruncatedGaussian',
 'end_date#0': 'copulas.univariate.gamma.GammaUnivariate',
 'end_date#1': 'copulas.univariate.gaussian.GaussianUnivariate',
 'experience_years': 'copulas.univariate.gaussian.GaussianUnivariate',
 'gender#0': 'copulas.univariate.gaussian.GaussianUnivariate',
 'gender#1': 'copulas.univariate.gaussian.GaussianUnivariate',
 'high_perc': 'copulas.univariate.log_laplace.LogLaplace',
 'high_spec#0': 'copulas.univariate.gamma.GammaUnivariate',
 'high_spec#1': 'copulas.univariate.gaussian.GaussianUnivariate',
 'high_spec#2': 'copulas.univariate.gaussian.GaussianUnivariate',
 'mba_perc': 'copulas.univariate.gamma.GammaUnivariate',
 'mba_spec#0': 'c

Conditional sampling

In [45]:
conditions = {
    'gender': 'M',
    'experience_years': 0
}
model.sample(10, conditions=conditions)

,student_id,gender,second_perc,high_perc,high_spec,degree_perc,degree_type,work_experience,experience_years,employability_perc,mba_spec,mba_perc,salary,placed,start_date,end_date,duration
0,17303,M,58.81,74.83,Commerce,64.89,Comm&Mgmt,False,0,51.00,Mkt&Fin,56.69,24500.0,True,2020-02-05,2021-01-10,12.0
1,17453,M,78.81,65.58,Arts,57.92,Comm&Mgmt,False,0,63.97,Mkt&Fin,63.05,31900.0,True,2020-01-17,2020-10-15,12.0
2,17411,M,60.51,65.91,Commerce,60.58,Comm&Mgmt,False,0,64.87,Mkt&Fin,61.21,31500.0,True,2020-01-18,2020-06-25,NaN
3,17278,M,70.22,61.86,Commerce,62.34,Comm&Mgmt,False,0,85.34,Mkt&Fin,58.70,34700.0,True,2020-03-16,2020-05-25,3.0
4,17384,M,80.71,56.50,Science,76.42,Sci&Tech,False,0,70.64,Mkt&HR,74.94,27600.0,True,2020-02-16,2020-10-05,12.0
5,17290,M,79.19,70.08,Science,63.71,Sci&Tech,False,0,82.46,Mkt&Fin,51.21,28500.0,True,2020-03-06,2020-07-18,3.0
6,17302,M,81.65,71.26,Science,81.08,Sci&Tech,False,0,86.15,Mkt&HR,61.83,27200.0,True,2020-02-07,2021-03-17,12.0
7,17440,M,64.20,42.49,Commerce,70.35,Comm&Mgmt,False,0,54.97,Mkt&Fin,55.77,28800.0,True,2020-01-06,2020-05-29,3.0
8,17392,M,82.57,71.47,Commerce,69.01,Comm&Mgmt,False,0,96.12,Mkt&Fin,77.89,38500.0,True,2020-02-04,2020-05-12,3.0
9,17413,M,78.64,73.96,Commerce,59.51,Comm&Mgmt,False,0,65.70,Mkt&Fin,69.69,31400.0,True,2020-09-10,2020-10-04,3.0


Save model

In [ ]:
model.save('my_model.pkl')
# Load model
# loaded = GaussianCopula.load('my_model.pkl')

####**CTGAN**

Load original data

In [46]:
from sdv.demo import load_tabular_demo

data = load_tabular_demo('student_placements')
data.head()

,student_id,gender,second_perc,high_perc,high_spec,degree_perc,degree_type,work_experience,experience_years,employability_perc,mba_spec,mba_perc,salary,placed,start_date,end_date,duration
0,17264,M,67.00,91.00,Commerce,58.00,Sci&Tech,False,0,55.0,Mkt&HR,58.80,27000.0,True,2020-07-23,2020-10-12,3.0
1,17265,M,79.33,78.33,Science,77.48,Sci&Tech,True,1,86.5,Mkt&Fin,66.28,20000.0,True,2020-01-11,2020-04-09,3.0
2,17266,M,65.00,68.00,Arts,64.00,Comm&Mgmt,False,0,75.0,Mkt&Fin,57.80,25000.0,True,2020-01-26,2020-07-13,6.0
3,17267,M,56.00,52.00,Science,52.00,Sci&Tech,False,0,66.0,Mkt&HR,59.43,NaN,False,NaT,NaT,NaN
4,17268,M,85.80,73.60,Commerce,73.30,Comm&Mgmt,False,0,96.8,Mkt&Fin,55.50,42500.0,True,2020-07-04,2020-09-27,3.0


Constraints (reject sampling)

In [50]:
from sdv.constraints import GreaterThan

end_date_gt_start_date_constraint = GreaterThan(
    low='start_date',
    high='end_date',
    handling_strategy='reject_sampling'
)

Fit original data

In [51]:
from sdv.tabular import CTGAN

constraints = [
    end_date_gt_start_date_constraint
]

model = CTGAN(constraints=constraints)
model.fit(data)

/usr/local/lib/python3.7/dist-packages/scipy/stats/_continuous_distns.py:547: RuntimeWarning: invalid value encountered in sqrt
  sk = 2*(b-a)*np.sqrt(a + b + 1) / (a + b + 2) / np.sqrt(a*b)
/usr/local/lib/python3.7/dist-packages/scipy/stats/_continuous_distns.py:4798: RuntimeWarning: divide by zero encountered in true_divide
  return c**2 / (c**2 - n**2)
/usr/local/lib/python3.7/dist-packages/scipy/stats/_distn_infrastructure.py:2407: RuntimeWarning: invalid value encountered in double_scalars
  Lhat = muhat - Shat*mu
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:72: FutureWarning: Pass n_components=10 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  "will result in an error", FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/mixture/_base.py:269: ConvergenceWarning: Initialization 1 did not converge. Try different init parameters, or increase max_iter, tol or check for degenerate data

Conditional sampling

In [52]:
conditions = {
    'gender': 'M',
    'experience_years': 1
}
model.sample(10, conditions=conditions)

,student_id,gender,second_perc,high_perc,high_spec,degree_perc,degree_type,work_experience,experience_years,employability_perc,mba_spec,mba_perc,salary,placed,start_date,end_date,duration
0,17336,M,40.89,94.44,Science,63.68,Comm&Mgmt,True,1,70.65,Mkt&Fin,53.05,30500.0,True,2020-03-01,2020-11-25,3.0
1,17345,M,77.73,37.00,Commerce,66.78,Sci&Tech,False,1,98.00,Mkt&Fin,63.16,45100.0,True,2020-03-18,2020-08-03,3.0
2,17283,M,40.89,97.70,Commerce,59.99,Sci&Tech,False,1,62.39,Mkt&HR,51.21,30300.0,False,2020-02-21,2020-08-08,12.0
3,17323,M,69.26,64.32,Commerce,81.93,Sci&Tech,False,1,63.23,Mkt&Fin,72.56,NaN,False,2019-12-17,2020-08-05,NaN
4,17368,M,50.83,82.64,Commerce,87.57,Comm&Mgmt,True,1,60.44,Mkt&HR,65.61,20900.0,False,2020-02-24,2020-03-20,3.0
5,17279,M,50.29,63.93,Science,53.42,Comm&Mgmt,True,1,53.76,Mkt&Fin,60.67,NaN,True,2020-05-01,2020-05-10,NaN
6,17424,M,84.69,70.20,Science,55.06,Comm&Mgmt,True,1,69.73,Mkt&HR,60.93,20000.0,False,2019-12-27,2020-07-07,3.0
7,17414,M,67.14,68.79,Science,68.42,Comm&Mgmt,True,1,50.00,Mkt&HR,76.40,NaN,True,2020-03-20,2020-08-24,NaN
8,17474,M,84.00,65.54,Commerce,75.96,Comm&Mgmt,True,1,98.00,Mkt&Fin,57.75,NaN,True,2020-01-30,2020-12-31,NaN
9,17371,M,84.53,46.38,Science,66.66,Comm&Mgmt,False,1,60.43,Mkt&HR,74.63,NaN,False,2020-01-05,2020-07-10,3.0


In [53]:
import pandas as pd 

conditions = pd.DataFrame({
    'gender': ['M', 'M', 'M', 'F', 'F', 'F'],
    'work_experience': [True, True, True, True, True, True]
})
model.sample(conditions=conditions)

,student_id,gender,second_perc,high_perc,high_spec,degree_perc,degree_type,work_experience,experience_years,employability_perc,mba_spec,mba_perc,salary,placed,start_date,end_date,duration
0,17309,M,40.89,97.70,Commerce,56.70,Comm&Mgmt,True,0,60.69,Mkt&HR,54.24,27000.0,False,2019-12-18,2020-08-06,3.0
1,17478,M,64.47,43.51,Commerce,59.48,Comm&Mgmt,True,0,50.00,Mkt&Fin,67.13,23900.0,True,2020-01-28,2020-09-28,3.0
2,17264,M,82.37,66.97,Commerce,87.80,Comm&Mgmt,True,0,59.46,Mkt&HR,72.76,NaN,True,2020-06-23,2020-08-28,NaN
3,17264,F,49.98,92.42,Science,56.86,Sci&Tech,True,0,50.00,Mkt&HR,68.97,41900.0,False,2020-06-24,2020-11-20,NaN
4,17379,F,85.18,37.00,Commerce,60.38,Sci&Tech,True,0,68.78,Mkt&Fin,69.58,37100.0,False,2020-03-12,2020-05-03,3.0
5,17350,F,65.51,68.33,Commerce,72.06,Sci&Tech,True,0,52.18,Mkt&Fin,68.62,NaN,True,2020-02-05,2020-09-27,12.0


#### **TVAE**

Load original data

In [54]:
from sdv.demo import load_tabular_demo

data = load_tabular_demo('student_placements')
data.head()

,student_id,gender,second_perc,high_perc,high_spec,degree_perc,degree_type,work_experience,experience_years,employability_perc,mba_spec,mba_perc,salary,placed,start_date,end_date,duration
0,17264,M,67.00,91.00,Commerce,58.00,Sci&Tech,False,0,55.0,Mkt&HR,58.80,27000.0,True,2020-07-23,2020-10-12,3.0
1,17265,M,79.33,78.33,Science,77.48,Sci&Tech,True,1,86.5,Mkt&Fin,66.28,20000.0,True,2020-01-11,2020-04-09,3.0
2,17266,M,65.00,68.00,Arts,64.00,Comm&Mgmt,False,0,75.0,Mkt&Fin,57.80,25000.0,True,2020-01-26,2020-07-13,6.0
3,17267,M,56.00,52.00,Science,52.00,Sci&Tech,False,0,66.0,Mkt&HR,59.43,NaN,False,NaT,NaT,NaN
4,17268,M,85.80,73.60,Commerce,73.30,Comm&Mgmt,False,0,96.8,Mkt&Fin,55.50,42500.0,True,2020-07-04,2020-09-27,3.0


Constraints (reject sampling)

In [55]:
from sdv.constraints import GreaterThan

end_date_gt_start_date_constraint = GreaterThan(
    low='start_date',
    high='end_date',
    handling_strategy='reject_sampling'
)

Fit original data

In [56]:
from sdv.tabular import TVAE

constraints = [
    end_date_gt_start_date_constraint
]

model = TVAE(constraints=constraints)
model.fit(data)

/usr/local/lib/python3.7/dist-packages/scipy/stats/_continuous_distns.py:547: RuntimeWarning: invalid value encountered in sqrt
  sk = 2*(b-a)*np.sqrt(a + b + 1) / (a + b + 2) / np.sqrt(a*b)
/usr/local/lib/python3.7/dist-packages/scipy/stats/_continuous_distns.py:4798: RuntimeWarning: divide by zero encountered in true_divide
  return c**2 / (c**2 - n**2)
/usr/local/lib/python3.7/dist-packages/scipy/stats/_distn_infrastructure.py:2407: RuntimeWarning: invalid value encountered in double_scalars
  Lhat = muhat - Shat*mu
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:72: FutureWarning: Pass n_components=10 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  "will result in an error", FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/mixture/_base.py:269: ConvergenceWarning: Initialization 1 did not converge. Try different init parameters, or increase max_iter, tol or check for degenerate data

Conditional sampling

In [59]:
conditions = {
    'gender': 'M',
    'experience_years': 0
}
model.sample(10, conditions=conditions)

,student_id,gender,second_perc,high_perc,high_spec,degree_perc,degree_type,work_experience,experience_years,employability_perc,mba_spec,mba_perc,salary,placed,start_date,end_date,duration
0,17397,M,74.17,74.68,Science,80.16,Comm&Mgmt,False,0,76.14,Mkt&Fin,74.41,25800.0,True,2020-01-18,2020-07-25,3.0
1,17292,M,83.95,76.46,Commerce,82.80,Comm&Mgmt,False,0,79.34,Mkt&Fin,63.64,23700.0,True,2020-01-14,2020-03-26,3.0
2,17297,M,85.11,57.98,Commerce,78.16,Comm&Mgmt,False,0,51.77,Mkt&Fin,62.30,21200.0,True,2020-01-23,2020-08-09,3.0
3,17445,M,72.79,72.62,Commerce,68.74,Comm&Mgmt,False,0,62.87,Mkt&Fin,61.90,27100.0,True,2020-01-21,2020-08-03,3.0
4,17416,M,67.04,77.87,Commerce,74.79,Comm&Mgmt,True,0,61.19,Mkt&Fin,67.88,23100.0,True,2020-01-20,2020-07-27,3.0
5,17473,M,83.28,65.55,Commerce,77.62,Comm&Mgmt,True,0,59.75,Mkt&Fin,61.90,21100.0,True,2020-01-15,2020-08-21,3.0
6,17449,M,68.12,60.75,Commerce,59.57,Comm&Mgmt,False,0,55.00,Mkt&HR,55.66,22600.0,True,2020-01-27,2020-08-09,6.0
7,17419,M,56.69,63.76,Commerce,65.04,Comm&Mgmt,False,0,53.01,Mkt&Fin,57.22,27300.0,True,2020-01-11,2020-08-03,NaN
8,17440,M,61.91,65.18,Commerce,65.97,Comm&Mgmt,False,0,59.80,Mkt&HR,56.60,22900.0,True,2020-01-24,2020-08-02,6.0
9,17296,M,78.98,78.10,Commerce,78.69,Comm&Mgmt,False,0,81.80,Mkt&Fin,77.02,24700.0,True,2020-01-16,2020-05-08,3.0


In [58]:
import pandas as pd 

conditions = pd.DataFrame({
    'gender': ['M', 'M', 'M', 'F', 'F', 'F'],
    'work_experience': [True, True, True, True, True, True]
})
model.sample(conditions=conditions)

,student_id,gender,second_perc,high_perc,high_spec,degree_perc,degree_type,work_experience,experience_years,employability_perc,mba_spec,mba_perc,salary,placed,start_date,end_date,duration
0,17468,M,82.26,76.60,Science,77.55,Comm&Mgmt,True,0,90.09,Mkt&Fin,69.02,24000.0,True,2020-01-20,2020-04-28,3.0
1,17419,M,82.54,81.26,Commerce,66.11,Comm&Mgmt,True,1,83.36,Mkt&Fin,67.70,24100.0,True,2020-01-25,2020-07-23,3.0
2,17438,M,77.12,69.91,Commerce,64.45,Comm&Mgmt,True,0,78.34,Mkt&Fin,75.05,24100.0,True,2020-01-06,2020-08-12,3.0
3,17349,F,71.28,71.05,Science,76.19,Sci&Tech,True,0,82.48,Mkt&Fin,67.17,23500.0,True,2020-01-17,2020-04-18,3.0
4,17399,F,82.78,68.54,Commerce,66.79,Comm&Mgmt,True,0,59.50,Mkt&Fin,64.06,21700.0,True,2020-02-04,2020-08-26,3.0
5,17311,F,87.53,78.38,Science,65.35,Sci&Tech,True,0,98.00,Mkt&Fin,68.59,28500.0,True,2020-01-07,2020-07-28,3.0
